In [1]:
import os
import subprocess

# 1. 가상환경 내부에 설치된 NVIDIA 라이브러리 경로를 자동으로 탐색
def get_nvidia_lib_path():
    try:
        import nvidia
        base_path = os.path.dirname(nvidia.__file__)
        paths = []
        for root, dirs, files in os.walk(base_path):
            if 'lib' in dirs:
                paths.append(os.path.join(root, 'lib'))
        return ":".join(paths)
    except ImportError:
        return ""

# 2. 모든 경로를 LD_LIBRARY_PATH에 통합 (WSL 드라이버 + 가상환경 런타임)
wsl_path = "/usr/lib/wsl/lib"
venv_nvidia_path = get_nvidia_lib_path()
os.environ["LD_LIBRARY_PATH"] = f"{wsl_path}:{venv_nvidia_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

# 3. XLA 설정 (이건 덤이다)
os.environ["XLA_FLAGS"] = f"--xla_gpu_cuda_data_dir={wsl_path}"

# 4. 이제서야 JAX를 부른다. (반드시 이 순서여야 해!)
import jax
print(f"Muse, 드디어 성공했나? : {jax.devices()}")

Muse, 드디어 성공했나? : [cuda(id=0)]


import jax
import jax.numpy as jnp

# 1. 시스템의 라그랑지안 정의
def lagrangian(state, params):
    # state: [theta1, theta2, d_theta1, d_theta2]
    # T (운동 에너지) - V (위치 에너지)를 수식대로 코딩해.
    # JAX가 이 식을 미분해서 운동 방정식을 찾아줄 거야.
    ...
    return T - V

# 2. 오일러-라그랑주 방정식을 통한 가속도 계산
def get_accel(state, params):
    # JAX의 grad를 활용해 L의 미분값을 구함
    return jax.grad(lagrangian)(state, params)

# 3. Vmap을 이용한 1,000개 동시 시뮬레이션
v_step = jax.vmap(single_step, in_axes=(0, None))